## kNN Classifier

In [21]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import torchvision
import torchvision.transforms as transforms
import warnings

In [22]:
# Load CIFAR-10 dataset using PyTorch
transform = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Prepare data for kNN (flatten images)
X_train = trainset.data.reshape(trainset.data.shape[0], -1).astype(np.float32)
y_train = np.array(trainset.targets)
X_test = testset.data.reshape(testset.data.shape[0], -1).astype(np.float32)
y_test = np.array(testset.targets)

# Normalize the data
X_train /= 255.0
X_test /= 255.0

# Split into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [23]:
# Function to evaluate kNN with different hyperparameters
def evaluate_knn(k_values, distance_metrics, X_train, y_train, X_val, y_val):
    results = {}
    for k in k_values:
        for metric in distance_metrics:
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric, algorithm='auto')
            knn.fit(X_train, y_train)
            val_acc = knn.score(X_val, y_val)
            results[(k, metric)] = val_acc
            print(f"k={k}, metric={metric}: Validation Accuracy = {val_acc:.4f}")
    return results

# Hyperparameter tuning for kNN
k_values = [1, 3, 5, 10, 20]
distance_metrics = ['l2']
knn_results = evaluate_knn(k_values, distance_metrics, X_train_split, y_train_split, X_val, y_val)

k=1, metric=l2: Validation Accuracy = 0.3357
k=3, metric=l2: Validation Accuracy = 0.3215
k=5, metric=l2: Validation Accuracy = 0.3325
k=10, metric=l2: Validation Accuracy = 0.3292
k=20, metric=l2: Validation Accuracy = 0.3180


In [25]:
# Cross-validation for kNN with optimizations
def cross_validate_knn(k_values, distance_metrics, X, y, cv=3):
    results = {}
    # Use a subset of data for faster cross-validation
    subset_size = 5000  # Reduce this if still too slow
    X_subset, _, y_subset, _ = train_test_split(X, y, train_size=subset_size, random_state=42)

    for k in k_values:
        for metric in distance_metrics:
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric, algorithm='auto')
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                # Use fewer folds and subset of data
                scores = cross_val_score(knn, X_subset, y_subset, cv=cv, n_jobs=-1)
            results[(k, metric)] = np.mean(scores)
            print(f"k={k}, metric={metric}: Cross-Validation Accuracy = {np.mean(scores):.4f} (std={np.std(scores):.4f})")
    return results

cv_results = cross_validate_knn(k_values, distance_metrics, X_train, y_train, cv=3)

k=1, metric=l2: Cross-Validation Accuracy = 0.2590 (std=0.0091)
k=3, metric=l2: Cross-Validation Accuracy = 0.2568 (std=0.0048)
k=5, metric=l2: Cross-Validation Accuracy = 0.2546 (std=0.0047)
k=10, metric=l2: Cross-Validation Accuracy = 0.2620 (std=0.0046)
k=20, metric=l2: Cross-Validation Accuracy = 0.2650 (std=0.0037)


In [26]:
# Find the best hyperparameters
best_params = max(cv_results, key=cv_results.get)
best_accuracy = cv_results[best_params]
print(f"\nBest hyperparameters: k={best_params[0]}, metric={best_params[1]}")
print(f"Best cross-validation accuracy: {best_accuracy:.4f}")


Best hyperparameters: k=20, metric=l2
Best cross-validation accuracy: 0.2650


## Linear Classifier, SVM Loss, and Softmax Loss

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

X_train = trainset.data.reshape(trainset.data.shape[0], -1).astype(np.float32)
y_train = np.array(trainset.targets)
X_test = testset.data.reshape(testset.data.shape[0], -1).astype(np.float32)
y_test = np.array(testset.targets)

# Normalize the data
X_train /= 255.0
X_test /= 255.0

# Add bias term
X_train = np.hstack((X_train, np.ones((X_train.shape[0], 1))))
X_test = np.hstack((X_test, np.ones((X_test.shape[0], 1))))

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LinearClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.fc(x)

# Multiclass SVM Loss
def svm_loss(scores, labels, delta=1.0):
    batch_size = scores.size(0)
    correct_class_scores = scores[torch.arange(batch_size), labels].view(-1, 1)
    margins = torch.max(torch.zeros_like(scores), scores - correct_class_scores + delta)
    margins[torch.arange(batch_size), labels] = 0
    loss = torch.mean(torch.sum(margins, dim=1))
    return loss

def softmax_loss(scores, labels):
    batch_size = scores.size(0)
    scores = scores - torch.max(scores, dim=1, keepdim=True).values
    exp_scores = torch.exp(scores)
    prob = exp_scores / torch.sum(exp_scores, dim=1, keepdim=True)
    loss = -torch.mean(torch.log(prob[torch.arange(batch_size), labels]))
    return loss

In [ ]:
input_size = X_train.shape[1]
num_classes = 10
model = LinearClassifier(input_size, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [19]:
# Training loop for SVM Loss
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    scores = model(X_train_tensor)
    loss = svm_loss(scores, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, SVM Loss: {loss.item():.4f}")

# Evaluate on test set using SVM Loss
model.eval()
test_scores = model(X_test_tensor)
svm_test_loss = svm_loss(test_scores, y_test_tensor)
print(f"SVM Test Loss: {svm_test_loss.item():.4f}")

Epoch 1, SVM Loss: 9.0546
Epoch 2, SVM Loss: 9.0468
Epoch 3, SVM Loss: 7.8521
Epoch 4, SVM Loss: 7.6262
Epoch 5, SVM Loss: 7.4045
Epoch 6, SVM Loss: 7.0860
Epoch 7, SVM Loss: 6.6570
Epoch 8, SVM Loss: 6.4565
Epoch 9, SVM Loss: 6.5708
Epoch 10, SVM Loss: 6.3879
SVM Test Loss: 6.0771


In [20]:
# Training loop for Softmax Loss
model = LinearClassifier(input_size, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    scores = model(X_train_tensor)
    loss = softmax_loss(scores, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Softmax Loss: {loss.item():.4f}")

# Evaluate on test set using Softmax Loss
model.eval()
test_scores = model(X_test_tensor)
softmax_test_loss = softmax_loss(test_scores, y_test_tensor)
print(f"Softmax Test Loss: {softmax_test_loss.item():.4f}")

Epoch 1, Softmax Loss: 2.3394
Epoch 2, Softmax Loss: 2.7214
Epoch 3, Softmax Loss: 2.4415
Epoch 4, Softmax Loss: 2.2715
Epoch 5, Softmax Loss: 2.3993
Epoch 6, Softmax Loss: 2.3230
Epoch 7, Softmax Loss: 2.2278
Epoch 8, Softmax Loss: 2.2289
Epoch 9, Softmax Loss: 2.2345
Epoch 10, Softmax Loss: 2.1771
Softmax Test Loss: 2.1165
